In [1]:
import pandas as pd
import numpy as np
tourney_df = pd.read_csv('data/TourneyDetailedResults.csv')
teams_df = pd.read_csv('data/Teams.csv')
reg_season_df = pd.read_csv('data/RegularSeasonCompactResults.csv')
team_name = dict(zip(teams_df['Team_Id'].values, teams_df['Team_Name'].values))
team_id = dict(zip(teams_df['Team_Name'].values, teams_df['Team_Id'].values))
tourney_df['Wteam_name'] = tourney_df['Wteam'].map(team_name)
tourney_df['Lteam_name'] = tourney_df['Lteam'].map(team_name)
Ngames = tourney_df.shape[0]
print("{0} tournament games in the dataset".format(Ngames))

847 tournament games in the dataset


# Randomizing the Winning Team

We first randomly chooses the winner to be team 1 or team 0.  The result is stored in `tourney_df('Winner')` and the ID corresponding to team0 and team1 are in `tourney_df('team0')` and `tourney_df('team1')`, respectively:

In [2]:
np.random.seed(0) # seed so we always get same answer
tourney_df['Winner'] = np.random.randint(2,size=Ngames)

def team0(game):
    if game['Winner'] == 1:
        team0 = game['Lteam']
        team1 = game['Wteam']
    else:
        team0 = game['Wteam']
        team1 = game['Lteam']    
    return team0

def team1(game):
    if game['Winner'] == 1:
        team0 = game['Lteam']
        team1 = game['Wteam']
    else:
        team0 = game['Wteam']
        team1 = game['Lteam'] 
    return team1

for feature in [team0, team1]:
    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)

tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Last,Lto,Lstl,Lblk,Lpf,Wteam_name,Lteam_name,Winner,team0,team1
0,2003,134,1421,92,1411,84,N,1,32,69,...,16,15,5,0,22,UNC Asheville,TX Southern,0,1421,1411
1,2003,136,1112,80,1436,51,N,0,31,66,...,12,17,10,3,15,Arizona,Vermont,1,1436,1112
2,2003,136,1113,84,1272,71,N,0,31,59,...,11,12,2,5,18,Arizona St,Memphis,1,1272,1113
3,2003,136,1141,79,1166,73,N,0,29,53,...,20,21,6,6,21,C Michigan,Creighton,0,1141,1166
4,2003,136,1143,76,1301,74,N,1,27,64,...,16,14,5,8,19,California,NC State,1,1301,1143


# ADD NEW FEATURES HERE (at bottom of cell below)

Add your new features here, see tutorial below

In [3]:
seasons = tourney_df['Season'].unique()
in_reg_season = dict(zip(seasons, [reg_season_df['Season'] == i for i in seasons]))
is_valid = reg_season_df['Wteam'] != 0

def wins0(game):  
    return reg_season_df.loc[(reg_season_df['Wteam'] == game['team0']) & in_reg_season[game['Season']] & is_valid].shape[0]
    
def wins1(game):  
    return reg_season_df.loc[(reg_season_df['Wteam'] == game['team1']) & in_reg_season[game['Season']] & is_valid].shape[0]    

In [4]:
fList = [wins0, wins1]
for feature in fList:
    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lstl,Lblk,Lpf,Wteam_name,Lteam_name,Winner,team0,team1,wins0,wins1
0,2003,134,1421,92,1411,84,N,1,32,69,...,5,0,22,UNC Asheville,TX Southern,0,1421,1411,13,18
1,2003,136,1112,80,1436,51,N,0,31,66,...,10,3,15,Arizona,Vermont,1,1436,1112,19,25
2,2003,136,1113,84,1272,71,N,0,31,59,...,2,5,18,Arizona St,Memphis,1,1272,1113,23,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,6,6,21,C Michigan,Creighton,0,1141,1166,23,29
4,2003,136,1143,76,1301,74,N,1,27,64,...,5,8,19,California,NC State,1,1301,1143,18,21


# Save the new data with features so we avoid regenerating

In [5]:
tourney_df.to_csv("data/data.csv", encoding='ascii')

# Generate Features for Submission Data

The cells below take the games Kaggle wants us to submit, and generates the corresponding features for them like we do above for the training data.

In [6]:
sample = pd.read_csv('data/SampleSubmission.csv')
sample.head()

,Id,Pred
0,2012_1104_1124,0.5
1,2012_1104_1125,0.5
2,2012_1104_1140,0.5
3,2012_1104_1143,0.5
4,2012_1104_1153,0.5


In [7]:
def get_season(Id):
    return np.int64(Id.split("_")[0])
def get_team0(Id):
    return np.int64(Id.split("_")[1])
def get_team1(Id):
    return np.int64(Id.split("_")[2])

sample['Season'] = sample['Id'].apply(get_season)
sample['team0'] = sample['Id'].apply(get_team0)
sample['team1'] = sample['Id'].apply(get_team1)

for feature in fList:
    sample[feature.__name__] = sample.apply(feature, axis=1)
    
sample.head()

,Id,Pred,Season,team0,team1,wins0,wins1
0,2012_1104_1124,0.5,2012,1104,1124,21,26
1,2012_1104_1125,0.5,2012,1104,1125,21,26
2,2012_1104_1140,0.5,2012,1104,1140,21,23
3,2012_1104_1143,0.5,2012,1104,1143,21,24
4,2012_1104_1153,0.5,2012,1104,1153,21,24


In [8]:
sample.to_csv("data/sample.csv", encoding='ascii')

# Tutorial for adding a new feature

All functions to populate a feature should take a game, i.e., a row of `tourney_df`,  

In [9]:
game = tourney_df.loc[0]
print(game)

Season                 2003
Daynum                  134
Wteam                  1421
Wscore                   92
Lteam                  1411
Lscore                   84
Wloc                      N
Numot                     1
Wfgm                     32
Wfga                     69
Wfgm3                    11
Wfga3                    29
Wftm                     17
Wfta                     26
Wor                      14
Wdr                      30
Wast                     17
Wto                      12
Wstl                      5
Wblk                      3
Wpf                      22
Lfgm                     29
Lfga                     67
Lfgm3                    12
Lfga3                    31
Lftm                     14
Lfta                     31
Lor                      17
Ldr                      28
Last                     16
Lto                      15
Lstl                      5
Lblk                      0
Lpf                      22
Wteam_name    UNC Asheville
Lteam_name      TX S

If it's a feature specific to a team, name it 0 or 1, and access team0's and/or team1's ID using the corresponding column.  `team_name` is a dictionary that takes a team ID and returns the name, and `team_id` is a dictionary that takes a team name and returns the ID.  You can use these to match up data you find elsewhere.

The function should return the value for your new feature.  As a simple example, to make a feature with the team's name, 

In [10]:
def name0(game):
    ID = game['team0']
    
    # Examples of using the dictionaries
    name = team_name[ID] 
    print(team_id[name]) 
    
    return name

name0(tourney_df.loc[0])

1421


'UNC Asheville'

# More Realistic Example

I had to look at different pandas examples/tutorials, but this might get you started.  For `wins0`, I wanted to be able to return the number of games `team0` won in the regular season.  What we actually have is a list of games, each of which have a winning team.  So we need to find all the games where `team0` was the winner, and add them all up.  So we start with the column that has all the IDs of the winning teams for each game.

In [11]:
wteam = reg_season_df['Wteam']
print(wteam.head())

0    1228
1    1106
2    1112
3    1165
4    1192
Name: Wteam, dtype: int64


Now we make a boolean array that has the same length as `wteam` above, with `True` if the winning team is `team0` (which we take to be 1106 in this example):

In [12]:
games_where_1106_won = reg_season_df['Wteam'] == 1106
print(games_where_1106_won.head())

0    False
1     True
2    False
3    False
4    False
Name: Wteam, dtype: bool


In [13]:
games_1106_won = wteam[games_where_1106_won]
print(games_1106_won.head())

1       1106
175     1106
1459    1106
1732    1106
2311    1106
Name: Wteam, dtype: int64


But this has games for all seasons mixed together, so we have to choose the right season.  Say the game for which we're trying to make the feature happend in 2012.  Then we need another mask like we used above to separate the games where 1106 won, but this time for the season:

In [14]:
in_2012 = reg_season_df['Season'] == 2012
print(in_2012.head())

0    False
1    False
2    False
3    False
4    False
Name: Season, dtype: bool


We can put boolean masks together like this:

In [15]:
games_1106_won_in_2012 = wteam[games_where_1106_won & in_2012]
print(games_1106_won_in_2012.head())

119676    1106
120814    1106
120893    1106
121126    1106
121296    1106
Name: Wteam, dtype: int64


For convenience, I've made a dictionary `in_reg_season` that takes a year, and gives you the corresponding boolean mask for that year.  Using that notation, and finally also using the `is_valid` mask, which ensures that the game is valid (WTeam != 0), we have 

In [16]:
games_1106_won_in_2012 = wteam[games_where_1106_won & in_reg_season[2012] & is_valid]
print(games_1106_won_in_2012.head())

119676    1106
120814    1106
120893    1106
121126    1106
121296    1106
Name: Wteam, dtype: int64


We can now get the length of the final array of filtered values using `shape`, and taking `[0]`, which gives the number of rows.  A minor final detail is that all these new variables make copies of slices of the dataframes, so it's slightly more efficient to do things without making copies with the `loc` function, which explains the one-liners in the wins0 and wins1 functions above: 

In [17]:
print(games_1106_won_in_2012.shape[0])
print(reg_season_df.loc[(reg_season_df['Wteam'] == 1106) & in_reg_season[2012] & is_valid].shape[0])

10
10
